#**Loading Data**

In [1]:
from io import StringIO
from google.colab import files

In [2]:
uploadedFiles = files.upload()

Saving stopWords.txt to stopWords.txt
Saving theBlue.txt to theBlue.txt


In [3]:
def read (fileName):
  return decode(fileName).read()

In [4]:
def decode(fileName):
  return StringIO(uploadedFiles[fileName + '.txt'].decode("utf-8"))

In [5]:
def getInput(txtType):
  return input("please enter " + txtType + " file name:   ")

In [6]:
file01 = read(getInput('text'))
stopwords = read(getInput('stop word'))

please enter text file name:   theBlue
please enter stop word file name:   stopWords


# **Deleting Stopwords**

In [7]:
import re
from nltk.tokenize import RegexpTokenizer

In [8]:
tokenizer = RegexpTokenizer('\s+', gaps = True)

In [9]:
def tokenizeText (text):
  return tokenizer.tokenize(re.sub("\W", " ", text))

In [10]:
stopwords = tokenizeText(stopwords)

In [11]:
def deleteStopWords(tokens):
  filtered_tokens = ' '
  
  for word in tokens:
    if word not in stopwords:
      filtered_tokens = filtered_tokens + ' ' + word

  return filtered_tokens

In [12]:
def makeSet (toBeSet):
  return set(deleteStopWords(tokenizeText(toBeSet)))

# **Apriori Algorithm**

In [13]:
import pandas as pd

In [14]:
def changeRowVal(tr, row, newVal):
  newVal = set(tokenizeText(newVal))
  return tr.replace(row, str(newVal), inplace = True)

In [15]:
def deleteEmptyRows(tr):
  return tr.drop(tr[tr.column == ' '].index, inplace = True)

In [16]:
def updateTransaction(tr):
  for row in tr.itertuples():
    changeRowVal(tr, row.column, deleteStopWords(tokenizeText(row.column)))
  
  return deleteEmptyRows(tr)

In [17]:
def findIt(isFirst, group, row):
  if isFirst:
    return group in row.column
  else:
    rowSet = set(tokenizeText(row.column))
    return set(group).intersection(rowSet) == set(group)

In [18]:
def findFreq(tr, theSet, isFirst):
  countItem = {}

  for group in theSet:
    freq = 0
    for row in tr.itertuples():
      if findIt(isFirst, group, row):
        freq = freq + 1
        
    countItem[group] = freq
  
  return countItem

In [19]:
def createTable(countItem):
  data = pd.DataFrame()

  data['itemset'] = countItem.keys()
  data['frequency'] = countItem.values()

  return data

In [20]:
def prune(data, supp):  
    return data[data.frequency >= supp]

In [21]:
def join(list_of_items):
    itemsets = []
    i = 1
    for entry in list_of_items:
        proceding_items = list_of_items[i:]
        for item in proceding_items:
            if(type(item) is str):
                if entry != item:
                    tuples = (entry, item)
                    itemsets.append(tuples)
            else:
                if entry[0:-1] == item[0:-1]:
                    tuples = entry+item[1:]
                    itemsets.append(tuples)
        i = i+1
    if(len(itemsets) == 0):
        return None
        
    return itemsets

In [22]:
def extractPattern(theSet, tr):
  for i in range(int(len(theSet)/ 2)):
    if i == 0:
      transT = createTable(findFreq(tr, theSet, 1))
    else:
      transT = createTable(findFreq(tr, set(itemsetList), 0))

    transT = prune(transT, 4)

    itemsetList = join(transT.itemset)
    if itemsetList == None:
      break

  return transT

#**Extracting Frequent Pattern of a Text**

In [23]:
file01_set = set(tokenizeText(deleteStopWords(tokenizeText(file01))))

In [24]:
transaction_df = pd.read_csv(decode("theBlue"), header = None)
transaction_df.columns = ['column']

In [25]:
updateTransaction(transaction_df)

transaction_df

,column
0,"{'آبی', 'یاد', 'پرانرژی', 'مهربان', 'هایی', 'ن..."
1,"{'گذاشتم', 'هرکجا', 'انگیز', 'بودم', 'می', 'دس..."
2,"{'دوستش', 'آبی', 'زیاد', 'زیبایی', 'تو', 'نداش..."
3,"{'حواسم', 'وسایل', 'ارزشش', 'آبی', 'خواستم', '..."
4,"{'حواسم', 'آبی', 'یاد', 'روز', 'بودم', 'بردی',..."
5,"{'حواسم', 'داشتم', 'آبی', 'شدم', 'ناامیدی', 'ه..."
6,"{'کمتر', 'حواسم', 'داشتم', 'کافی', 'آبی', 'بود..."
7,"{'شدی', 'روز', 'متنفر', 'زرد', 'بالاخره', 'تو'..."
8,"{'دنیای', 'آبی', 'خواهم', 'خودم', 'داشتنی', 'م..."


In [26]:
extractPattern(file01_set, transaction_df)

,itemset,frequency
9,"(دنیای, نمی, آبی)",4
12,"(دنیای, حواسم, آبی)",4
15,"(دنیای, می, آبی)",4
19,"(دنیای, تو, آبی)",4
21,"(دنیای, رنگ, آبی)",5
